In [5]:
from datetime import datetime
import pandas as pd
import praw
import json
import pyarrow as pa
import pyarrow.parquet as pq

from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()


def process_reddit_data():
    reddit = praw.Reddit(
        client_id=os.getenv("CLIENT_ID"),
        client_secret=os.getenv("SECRET_KEY"),
        user_agent=os.getenv("REDDIT_GRANT_TYPE"),
        username=os.getenv("REDDIT_USERNAME"),
        password=os.getenv("REDDIT_PASSWORD")
    )

    dfComment = pd.DataFrame(columns=['id', 'timestamp', 'author', 'body'])
    index_loop = 0

    submission_id = "isfwqm"
    submission = reddit.submission(submission_id)
    submission.comments.replace_more(limit=None)

    for comment in submission.comments.list():
        dfComment.loc[index_loop, 'id'] = comment.id
        dfComment.loc[index_loop, "timestamp"] = datetime.utcfromtimestamp(comment.created_utc)
        dfComment.loc[index_loop, 'author'] = str(comment.author)
        dfComment.loc[index_loop, 'body'] = comment.body
        index_loop += 1

    dfComment['dt'] = pd.to_datetime(dfComment['timestamp']).dt.strftime('%Y-%m-%d')
    dfComment['author'] = dfComment['author'].apply(lambda x: str(x))

    print(dfComment)

    tmp_dir = '/tmp/'
    ls_dt = sorted(list(dfComment['dt'].unique()))

    for dt in ls_dt:
        file_name = f'{submission_id}_{dt}.parquet'
        dfComment_daily = dfComment[dfComment['dt'] == dt]

        arrow_table = pa.Table.from_pandas(dfComment_daily)
        pq.write_table(arrow_table, tmp_dir + file_name)

        print(tmp_dir + file_name)

    return None

if __name__ == "__main__":
    process_reddit_data()

MissingRequiredAttributeException: Required configuration setting 'client_id' missing. 
This setting can be provided in a praw.ini file, as a keyword argument to the Reddit class constructor, or as an environment variable.

In [4]:
!pip install praw

  Using cached praw-7.7.1-py3-none-any.whl (191 kB)
  Using cached prawcore-2.4.0-py3-none-any.whl (17 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl (7.0 kB)
